In [7]:
import os
import cv2
import numpy as np
import tensorflow as tf
import os

2023-11-26 14:48:11.992754: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-26 14:48:12.031879: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-26 14:48:12.032666: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 14:48:12.678872: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
# Set the path to the dataset
dataset_path = '.'

# Set the paths to images and annotations
images_folder = os.path.join(dataset_path, 'Dataset')
annotations_folder = os.path.join(dataset_path, 'Bounding_Boxes')

# Set the number of classes
num_classes = 3

# Model configuration
input_shape = (224, 224, 3)  # Adjust based on your image size
batch_size = 32
epochs = 20

In [32]:
# Data preprocessing and loading
def load_data():
    images = []
    annotations = []
    label_map = {}
    label_id = 1

    # return images, labels
    for annotation_name in os.listdir(annotations_folder):

        # If no .txt files, continue
        if not annotation_name.endswith('.txt'):
            continue

        # Load image
        image_name = annotation_name.replace('.txt', '.jpg')
        image_path = os.path.join(images_folder, image_name)
        image = cv2.imread(image_path)
        image_width, image_height = image.shape[1], image.shape[0]
        image = cv2.resize(image, (input_shape[0], input_shape[1]))

        images.append(image)

        # Load annotations
        annotation_path = os.path.join(annotations_folder, annotation_name)
        with open(annotation_path, 'r') as f:
            # Parse bounding box coordinates and class label
            # Example format: class x_min y_min x_max y_max
            lines = f.readlines()
            sub_annotations = []
            for line in lines:
                line = line.strip().split(' ')
                class_label, x_min, y_min, x_max, y_max = map(float, line)

                # Convert normalized coordinates to absolute coordinates
                x_min *= image_width
                y_min *= image_height
                x_max *= image_width
                y_max *= image_height

                # Rescale coordinates to input_shape
                x_min = int(x_min * input_shape[0] / image_width)
                y_min = int(y_min * input_shape[1] / image_height)
                x_max = int(x_max * input_shape[0] / image_width)
                y_max = int(y_max * input_shape[1] / image_height)

                # Add class label to label map if not already present
                if class_label not in label_map:
                    label_map[class_label] = label_id
                    label_id += 1

                # Store annotation information
                sub_annotations.append({
                    'image_path': os.path.join(images_folder, f'{image_name}'),
                    'bbox': [x_min, y_min, x_max, y_max],
                    'class_label': label_map[class_label]
                })

            annotations.append(sub_annotations)

    images = np.array(images)

    return images, annotations


# Load and preprocess the data
images, annotations = load_data()

images.shape, len(annotations)


((2, 224, 224, 3), 2)

In [34]:
class_map = {
    1: 'category_title',
    2: 'menu_item',
    3: 'section'
}

In [35]:
annotations[0]


[{'image_path': './Dataset/image1.jpg',
  'bbox': [52, 90, 93, 56],
  'class_label': 1},
 {'image_path': './Dataset/image1.jpg',
  'bbox': [52, 132, 91, 27],
  'class_label': 1},
 {'image_path': './Dataset/image1.jpg',
  'bbox': [51, 161, 91, 29],
  'class_label': 1},
 {'image_path': './Dataset/image1.jpg',
  'bbox': [51, 186, 91, 19],
  'class_label': 1},
 {'image_path': './Dataset/image1.jpg',
  'bbox': [50, 206, 91, 19],
  'class_label': 1},
 {'image_path': './Dataset/image1.jpg',
  'bbox': [167, 20, 87, 22],
  'class_label': 1},
 {'image_path': './Dataset/image1.jpg',
  'bbox': [167, 41, 86, 20],
  'class_label': 1},
 {'image_path': './Dataset/image1.jpg',
  'bbox': [50, 68, 86, 8],
  'class_label': 2},
 {'image_path': './Dataset/image1.jpg',
  'bbox': [48, 122, 84, 6],
  'class_label': 2},
 {'image_path': './Dataset/image1.jpg',
  'bbox': [49, 152, 85, 8],
  'class_label': 2},
 {'image_path': './Dataset/image1.jpg',
  'bbox': [50, 183, 87, 8],
  'class_label': 2},
 {'image_path': 